# Numerical Techniques 5: Beyond the 1D advection equation



## Preparations: load numpy and matplotlib libraries

In [ ]:
## Load numpy and matplotlib libraries
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.ioff()
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 100  # reduce to generate figures faster (but smaller)

# function for animating results
def create_animation(phi,frames=None,labels=None,ylim=None):
    
    # create figure with axes
    fig,ax=plt.subplots()
    
    if len(phi.shape)==2:
        phi=phi.reshape([phi.shape[0],phi.shape[1],1])
    
    # number of experiments
    nExp=phi.shape[2]

    # determine frames to plot
    if frames is None:
        # number of timesteps
        nt=phi.shape[0]-1
        frames=np.arange(0,nt+1)
    else:
        nt=frames[-1]
    
    
    # set y limits
    if ylim is None:
        ymin=np.min(phi[0,:,:])
        ymax=np.max(phi[0,:,:])
        ylim=[1.2*ymin-0.2*ymax,1.2*ymax-0.2*ymin]
    
    # labels and legend   
    showlegend=True
    if labels is None:
        showlegend=False
        labels=['exp %i'%jExp for jExp in range(nExp)]
        
    # create lines for numerical solution
    ll_n=[ax.plot(x,phi[frames[0],:,jExp],label=labels[jExp])[0] for jExp in range(nExp)]
    
    # set ylim
    ax.set_ylim(ylim)
    
    # add legend
    if showlegend:
        plt.legend()
    
    # add title
    tt=plt.title('')
    

    def animate(iframe):
        for jExp in range(nExp):
            ll_n[jExp].set_ydata(phi[iframe,:,jExp])
        tt.set_text('timestep %i/%i'%(iframe,nt))

    anim=animation.FuncAnimation(fig, animate, frames=frames,cache_frame_data=False,blit=True)
    display(anim)

## Shallow water equations

The 1D linearized shallow water equations are given by

$$\begin{aligned}
\frac{\partial u}{\partial t}+U\frac{\partial u}{\partial x}+g\frac{\partial h}{\partial x} &=0 \\
\frac{\partial h}{\partial t}+H\frac{\partial u}{\partial x}+U\frac{\partial h}{\partial x} &=0
\end{aligned}$$

where the unknown fields are $u$ (horizontal velocity, superposed to $U$), $h$ (water height, superposed to $H$), and the fixed parameters are $U$ (background horizontal velocity), $H$ (background water height) and $g$ (gravity constant).

In [ ]:
# solving with leapfrog+2nd order centered differences

# parameters
dt=0.1
nt=300

dx=1
nx=128

U=1
H=1
g=9.81

# courant number
mu=(np.abs(U)+np.sqrt(g*H))*dt/dx
print('Courant number : ',mu)

# grid
x=np.arange(nx)*dx
t=np.arange(nt)*dt

# allocate solution
h=np.zeros((nt+1,nx))
u=np.zeros((nt+1,nx))

# initial state: gaussian bell for water height
h[0,:]=0.5*np.exp(-40*(x/(nx*dx)-0.5)**2)

# indices of points to the left and right
j=np.arange(nx)
jL=(j-1)%nx
jR=(j+1)%nx

# first timestep: forward scheme
jt=0
u[jt+1,:]=u[jt,:]-dt*(U*(u[jt,jR]-u[jt,jL])+g*(h[jt,jR]-h[jt,jL]))/(2*dx)
h[jt+1,:]=h[jt,:]-dt*(H*(u[jt,jR]-u[jt,jL])+U*(h[jt,jR]-h[jt,jL]))/(2*dx)

# later timesteps: leapfrog
for jt in range(1,nt):
    u[jt+1,:]=u[jt-1,:]-dt*(U*(u[jt,jR]-u[jt,jL])+g*(h[jt,jR]-h[jt,jL]))/dx
    h[jt+1,:]=h[jt-1,:]-dt*(H*(u[jt,jR]-u[jt,jL])+U*(h[jt,jR]-h[jt,jL]))/dx
    
# animate result (height)
create_animation(h,frames=np.arange(0,nt+1,4))


## Diffusion equation

The diffusion equation is given as

$$\frac{\partial\psi}{\partial t}=\frac{\partial}{\partial x}K\frac{\partial\psi}{\partial x}$$

Solving it with a forward scheme and 2nd order finite differences gives:

In [ ]:
# parameters
dt=0.1
nt=1500

nx=128
dx=2*np.pi/nx

K=0.01

# stability number
nu=K*dt/(dx**2)
print('nu : ',nu)

# grid
x=np.arange(nx)*dx
t=np.arange(nt)*dt

# allocate solution
phi=np.zeros((nt+1,nx))

# initial state: summation of harmonic functions + small-scale noise
phi[0,:]=np.cos(x)+np.sin(2*x)+np.cos(3*x)-np.cos(4*x)+np.random.rand(nx)

# indices of points to the left and right
j=np.arange(nx)
jL=(j-1)%nx
jR=(j+1)%nx

# first timestep: forward scheme
for jt in range(nt):
    phi[jt+1,:]=phi[jt,:]+dt*K*(phi[jt,jL]-2*phi[jt,:]+phi[jt,jR])/(dx**2)

# animate result
create_animation(phi,frames=np.arange(0,nt//50))

## Burger's equation

Burger's equation is a nonlinear advection equation, where the advection speed is given by the field itself:

$$\frac{\partial\psi}{\partial t}+\psi\frac{\partial\psi}{\partial x}=0$$

Let's integrate with the highly-accurate Runge-Kutta-4 and consider different spatial discretizations.

In [ ]:
# parameters
dt=0.1
nt=1500

nx=100
dx=1

# grid
x=np.arange(nx)*dx
t=np.arange(nt+1)*dt

# allocate solution
nExp=3
phi=np.zeros((nt+1,nx,nExp))

# initial state: summation of harmonic functions + small-scale noise
for jExp in range(nExp):
    phi[0,:,jExp]=np.sin(2*np.pi*x/(nx*dx))

# indices of points to the left and right
j=np.arange(nx)
jL=(j-1)%nx
jR=(j+1)%nx

# define right-hand side for different formulations
def rhs(phi):
    phi2=phi**2
    return( np.stack([
        -phi[:,0]*(phi[jR,0]-phi[jL,0])/(2*dx),     # advective form
        -(phi2[jR,1]-phi2[jL,1])/(4*dx),              # flux form
        -phi[:,2]*(phi[jR,2]-phi[jL,2])/(6*dx)-(phi2[jR,2]-phi2[jL,2])/(6*dx)   # conservative form
    ],1))

# integration with runge-kutta-4
for jt in range(nt):
    q1=dt*rhs(phi[jt])
    q2=dt*rhs(phi[jt]+q1/2)
    q3=dt*rhs(phi[jt]+q2/2)
    q4=dt*rhs(phi[jt]+q3)
    phi[jt+1]=phi[jt]+(q1+2*q2*2*q3+q4)/6
    # remove values that are too large
    for jExp in range(nExp):
        if not np.isnan(phi[jt,0,jExp]) and np.max(np.abs(phi[jt+1,:,jExp]))>1e6:
            phi[jt+1,:,jExp]=np.nan
# animate result
create_animation(phi[:,:,0],frames=np.arange(0,nt+1,5),labels=['advective'],ylim=[-3,3])

In [ ]:
create_animation(phi,frames=np.arange(0,nt+1,5),labels=['advective','flux','conservative'],ylim=[-3,3])